<a href="https://colab.research.google.com/github/iamjudy/deep-learning-colab/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [93]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dropout

### 1. 讀入 MNSIT 數據集

In [55]:
from tensorflow.keras.datasets import mnist

In [95]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

#### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [96]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255.

In [97]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255.

In [98]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)
##**把每個卷積層做兩次，並更改 filter 數量：10 -> 30 -> 60** 

In [100]:
model = Sequential()

In [101]:
model.add(Conv2D(10, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [102]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [103]:
model.add(Conv2D(30, (3,3), padding='same',
                activation='relu'))

In [104]:
model.add(Conv2D(30, (3,3), padding='same',
                activation='relu'))

In [105]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [106]:
model.add(Conv2D(60, (3,3), padding='same',
                activation='relu'))

In [107]:
model.add(Conv2D(60, (3,3), padding='same',
                activation='relu'))

In [108]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [109]:
model.add(Flatten())

## **輸出前砍掉 20%資料**

In [110]:
model.add(Dense(64, activation='relu'))

In [111]:
model.add(Dropout(0.2))

In [112]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [113]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 28, 10)        100       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 14, 10)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 14, 14, 30)        2730      
                                                                 
 conv2d_14 (Conv2D)          (None, 14, 14, 30)        8130      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 7, 7, 30)         0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 7, 7, 60)         

#### 組裝
## **optimizer 改用 Adam，調整 Learning Rate 為 0.001**

In [114]:
model.compile(loss='mse', optimizer=Adam(learning_rate=0.001),
             metrics=['accuracy'])

### 4. step 2: fit
## **更改 epochs 為 15**

In [115]:
model.fit(x_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
469/469 [==============================] - 7s 13ms/step - loss: 0.0144 - accuracy: 0.8974
Epoch 2/15
469/469 [==============================] - 6s 12ms/step - loss: 0.0038 - accuracy: 0.9758
Epoch 3/15
469/469 [==============================] - 6s 12ms/step - loss: 0.0028 - accuracy: 0.9822
Epoch 4/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0023 - accuracy: 0.9857
Epoch 5/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0019 - accuracy: 0.9885
Epoch 6/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0018 - accuracy: 0.9889
Epoch 7/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0016 - accuracy: 0.9898
Epoch 8/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0013 - accuracy: 0.9918
Epoch 9/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0012 - accuracy: 0.9922
Epoch 10/15
469/469 [==============================] - 5s 12ms/step - loss: 0.0012 - accura

### 5. Step 3: 預測

In [116]:
y_predict = np.argmax(model.predict(x_test), axis=-1)

#### 看看測試資料表現如何

In [117]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0011 - accuracy: 0.9934


## **耶！正確率有 99.34 ％**

In [118]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 99.34%


In [119]:
def my_predict(n):
    print('我可愛的 CNN 預測是', y_predict[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [120]:
from ipywidgets import interact_manual

In [121]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…